2. Sistema de clasificación de subreddit. 

Se deberá implementar una función ```classify_subreddit(text)``` que clasifique un texto de entrada en una de las siguientes 
categorías: **MachineLearning, datascience, statistics, learnmachinelearning, computerscience, AskStatistics, artificial, analytics, datasets, deeplearning, rstats, computervision, DataScienceJobs, MLQuestions, dataengineering, data, dataanalysis, 
datascienceproject, Kaggle**.

Para ello, se proporciona un dataset sobre el que se podrán entrenar distintos algoritmos
de clasificación. La etiqueta del subreddit correspondiente se encuentra en la columna "subreddit". Se deberán probar, al menos, los siguientes 3 métodos:
- Un método basado en TF-IDF + algoritmo de clasificación de machine learning
- Un método basado en entidades reconocidas (Named-Entity Recognition) + algoritmo declasificación de machine learning 
- Un método basado en Word Embeddings + algoritmo de clasificación de machine learning

Para evaluar cada método, se utilizará la métrica f1 score, y se utilizará un 70% de los datos
del dataset para entrenamiento y un 30% para test (realizando un sampling aleatorio
previo)."""

from IPython.display import Markdown
display(Markdown(ejercicio))

Los posts de entrenamiento se encuentran en la columna 'clean_post' del dataframe, y las labels en la columna 'subreddit'. Las siguientes son las categorías en las que clasificaremos cada post:

In [ ]:
categories = ['MachineLearning', 'datascience', 'statistics', 'learnmachinelearning', 'computerscience', 'AskStatistics', 'artificial', 'analytics', 'datasets', 'deeplearning', 'rstats', 'computervision', 'DataScienceJobs', 'MLQuestions', 'dataengineering', 'data', 'dataanalysis', 'datascienceproject', 'Kaggle']

In [1]:
import pandas as pd
from sklearn.feature_extraction import train_test_split

df = pd.read_csv('processed_dataset.csv')
x_data = df['clean_post']
y_data = df['subreddit']
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, )


KeyboardInterrupt



### Método basado en TF-IDF + algoritmo de clasificación de machine learning

<h4>Elección del algoritmo de machine learning</h4>

Para los métodos de clasificación usaremos regresión logística implementada en la librería sklearn por los siguientes motivos: 
<ul>
1. Ideal para clasificación binaria/multiclase

2. Interpretable: Los coeficientes indican importancia de features

3. Eficiente con datasets grandes

4. Bueno con datos dispersos (sparse) como text data

5. Rápido de entrenar y predecir
</ul>

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

# Define features and labels
X = df['clean_post']
y = df['subreddit']

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Transform the data
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Initialize and train classifier
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_tfidf, y_train)

# Predict and evaluate
y_pred = clf.predict(X_test_tfidf)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f'F1 Score: {f1}')

### Un método basado en entidades reconocidas (Named-Entity Recognition) + algoritmo declasificación de machine learning 

In [ ]:
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

# Load spaCy's English model
nlp = spacy.load('en_core_web_sm')

# Function to extract named entity labels
def extract_entity_labels(text):
    doc = nlp(text)
    return ' '.join([ent.label_ for ent in doc.ents])

# Extract entity labels from training and testing data
X_train_entities = X_train.apply(extract_entity_labels)
X_test_entities = X_test.apply(extract_entity_labels)

# Initialize CountVectorizer
vectorizer_entities = CountVectorizer()

# Transform the entity labels
X_train_entities_vectorized = vectorizer_entities.fit_transform(X_train_entities)
X_test_entities_vectorized = vectorizer_entities.transform(X_test_entities)

# Initialize and train classifier
clf_entities = LogisticRegression(max_iter=1000)
clf_entities.fit(X_train_entities_vectorized, y_train)

# Predict and evaluate
y_pred_entities = clf_entities.predict(X_test_entities_vectorized)
f1_entities = f1_score(y_test, y_pred_entities, average='weighted')
print(f'F1 Score (NER-based): {f1_entities}')

### Un método basado en Word Embeddings + algoritmo de clasificación de machine learning